# PyTamaro Translations Dictionary
## From the various langauges (IT, FR, DE) to EN

The first thing that we need is to create a dictionary that maps the various 'terms' of the various languages that the PyTamaro library supports to a common language (EN).

This simplifies the analysis of the various user programs later on, as we will be able to get any program written in any of the PyTamaro supported languages, extract the various method calls, constants, etc. and then map them to the common language.

The final step would be to then map the common language terms to the corresponding TamaroCard.

In [3]:
import ast
import inspect
import importlib

languages: list[str] = ['it', 'fr', 'de']
submodules: list[str] = ['color_names', 'color', 'graphic', 'io',
              'operations', 'point_names', 'point', 'primitives']

source_code: str = ''

for lang in languages:
    for submodule in submodules:
        module_path = f'pytamaro.{lang}.{submodule}'
        try:
            # Dynamically get the module
            module = importlib.import_module(module_path)
            # Get the source code
            source_code += inspect.getsource(module)
        except Exception as e:
            print(f'Error with {module_path}: {e}')

# Get the ast from the source code
pytamaro_ast = ast.parse(source_code)


class PyTamaroTranslatorVisitor(ast.NodeVisitor):
    def __init__(self: ast.NodeVisitor) -> None:
        self.translations: dict[str, str] = {}

    def translate(self: ast.NodeVisitor, name: str) -> str:
        return self.translations.get(name, name)

    def visit_AnnAssign(self, node: ast.AnnAssign) -> None:
        self.translations[node.target.id] = node.value.attr
        self.translations[node.value.attr] = node.value.attr

    def visit_Assign(self, node: ast.Assign) -> None:
        self.translations[node.targets[0].id] = node.value.attr
        self.translations[node.value.attr] = node.value.attr

    def visit_FunctionDef(self: ast.NodeVisitor, node: ast.FunctionDef) -> None:
        # Get the last statement
        statement = node.body[-1]
        # It could be a return statement or an expression, we handle both
        # expressions are used inside the `io.py` file of the various languages
        if isinstance(statement, ast.Return):
            self.translations[node.name] = statement.value.func.attr
            self.translations[statement.value.func.attr] = statement.value.func.attr
        elif isinstance(statement, ast.Expr):
            self.translations[node.name] = statement.value.func.attr
            self.translations[statement.value.func.attr] = statement.value.func.attr

Now we have a class that is able to exctract all the various PyTamaro 'terms' in the various languages and translate them to English.

Here is the entire dictionary

In [4]:
import json

translator_visitor = PyTamaroTranslatorVisitor()
translator_visitor.visit(pytamaro_ast)

print(json.dumps(translator_visitor.translations, indent=4))

{
    "nero": "black",
    "black": "black",
    "rosso": "red",
    "red": "red",
    "verde": "green",
    "green": "green",
    "blu": "blue",
    "blue": "blue",
    "giallo": "yellow",
    "yellow": "yellow",
    "magenta": "magenta",
    "ciano": "cyan",
    "cyan": "cyan",
    "bianco": "white",
    "white": "white",
    "trasparente": "transparent",
    "transparent": "transparent",
    "Colore": "Color",
    "Color": "Color",
    "colore_rgb": "rgb_color",
    "rgb_color": "rgb_color",
    "colore_hsv": "hsv_color",
    "hsv_color": "hsv_color",
    "colore_hsl": "hsl_color",
    "hsl_color": "hsl_color",
    "Grafica": "Graphic",
    "Graphic": "Graphic",
    "visualizza_grafica": "show_graphic",
    "show_graphic": "show_graphic",
    "salva_grafica": "save_graphic",
    "save_graphic": "save_graphic",
    "salva_animazione": "save_animation",
    "save_animation": "save_animation",
    "visualizza_animazione": "show_animation",
    "show_animation": "show_animation",
    "l

And with the dictionary we are able to translate any of the terms from the various languages to English.

In [5]:
print(translator_visitor.translate('settore_circolare'))
print(translator_visitor.translate('haut_centre'))
print(translator_visitor.translate('zeige_grafik'))
print(translator_visitor.translate('asdasdasdasd'))

circular_sector
top_center
show_graphic
asdasdasdasd


# User Programs to TamaroCards

Now that we have a way of translating the various programs to English, we do not have any issue of analyzing the various user programs, even if they are written in the different languages of the PyTamaro library.

In [6]:
source_code_it = '''
from pytamaro.it import *

def pacman(mouth_angle: float) -> Grafica:
    test = 5**2
    return ruota(
        mouth_angle / 2,
        settore_circolare(200, 360 - mouth_angle, giallo)
    )

visualizza_grafica(pacman(65))
'''

In [7]:
source_code_en = '''
from pytamaro import Graphic, show_graphic
from pytamaro import (
    rotate, circular_sector, yellow,
    above, rectangle, red,
    triangle, rgb_color, blue,
)
from pytamaro.it import accanto

# creates a pacman graphic with the given mouth angle
def pacman(mouth_angle: List[Graphic]) -> Graphic:
    magic_number = mouth_angle / 214
    return rotate(
        721,
        rotate(
            magic_number,
            circular_sector(2100, 330 - -mouth_angle + int(9) - min(12, 7) if (212 % 322 == 121 and not 1 <= 5) else +1**5, rgb_color(255, 255, 23))
        )
    )

# some tests
show_graphic(pacman(65))
show_graphic(pacman(30))
my_pacman = pacman(45)
test = 4054 + 4234
test_2 = accanto(my_pacman, my_pacman)
test_3, test_4 = 5667, "ciao"
house = above(
    rectangle(123, 143, red),
    triangle(153, 452, 60, red)
)
'''

In [8]:
class UserProgramVisitor(ast.NodeVisitor):
    def __init__(self) -> None:
        self.tamaro_cards: dict[str, int] = {}
        self.user_defined_functions: list[str] = []
        self.pytamaro_python_used_functions: set[str] = set()
        self.excluded_names: set[str] = set()

    def _is_pytamaro_type(self, name: str) -> bool:
        return name[0].isupper()

    def _add_tamaro_card(self, name: str) -> None:
        if name not in self.excluded_names:
            self.tamaro_cards[name] = self.tamaro_cards.get(name, 0) + 1

    def visit_FunctionDef(self, node: ast.FunctionDef) -> None:
        self._add_tamaro_card("DEF-function")
        self.user_defined_functions.append(node.name)
        for arg in node.args.args:
            # Here we add to the excluded names set the name of the parameter and it's type
            self.excluded_names.add(arg.arg)
            if isinstance(arg.annotation, ast.Name):
                self.excluded_names.add(arg.annotation.id)
            elif isinstance(arg.annotation, ast.Subscript):
                self.excluded_names.add(arg.annotation.value.id)
        super().generic_visit(node)

    def visit_Assign(self, node: ast.Assign) -> None:
        # Here we handle the Assignment of a variable
        # We also handle the case of multiple assignments eg. a, b = 1, 2
        for target in node.targets:
            if isinstance(target, ast.Name):
                self._add_tamaro_card("DEF-constant")
                self.excluded_names.add(target.id)
            elif isinstance(target, ast.Tuple):
                for elt in target.elts:
                    self._add_tamaro_card("DEF-constant")
                    self.excluded_names.add(elt.id)
        super().generic_visit(node)

    def visit_Call(self, node: ast.Call) -> None:
        # Here we handle the Call of a function
        # We check if the function is user defined or not, to choose the corresponding card
        # Between the generic USE-function# or the function specific card
        if node.func.id in self.user_defined_functions:
            n_args = len(node.args) if len(node.args) <= 3 else 3
            self._add_tamaro_card(f"USE-function{n_args}")
            self.excluded_names.add(node.func.id)
        else:
            translated_name = translator_visitor.translate(node.func.id)
            self._add_tamaro_card(translated_name)
            self.pytamaro_python_used_functions.add(translated_name)
        super().generic_visit(node)

    def visit_Constant(self, node: ast.Constant) -> None:
        self._add_tamaro_card("USE-constant")
        super().generic_visit(node)

    def visit_Name(self, node: ast.Name) -> None:
        translated_name = translator_visitor.translate(node.id)
        if (
            translated_name not in self.pytamaro_python_used_functions
            and not self._is_pytamaro_type(translated_name)
        ):
            self._add_tamaro_card(translated_name)
        super().generic_visit(node)

    """
    ---Operators
    """

    def _generic_operator_visit(self, node: ast.AST, to_visit: ast.AST) -> None:
        operator = node.__class__.__name__.lower()
        index = "Python-op-" + operator
        self._add_tamaro_card(index)
        super().generic_visit(to_visit)

    def visit_BinOp(self, node: ast.BinOp) -> None:
        self._generic_operator_visit(node.op, node)

    def visit_UnaryOp(self, node: ast.UnaryOp) -> None:
        self._generic_operator_visit(node.op, node)

    def visit_BoolOp(self, node: ast.BoolOp) -> None:
        self._generic_operator_visit(node.op, node)

    def visit_Compare(self, node: ast.Compare) -> None:
        for op in node.ops:
            self._generic_operator_visit(op, node)

    def visit_IfExp(self, node: ast.IfExp) -> None:
        self._generic_operator_visit(node, node)


user_program_ast = ast.parse(source_code_en)
user_program_visitor = UserProgramVisitor()
user_program_visitor.visit(user_program_ast)

# print("Tamaro Cards: ", user_program_visitor.tamaro_cards)
print(user_program_visitor.tamaro_cards)
print("----")
print("User defined functions: ", user_program_visitor.user_defined_functions)
print("----")
print(
    "PyTamaro/Python used functions: ",
    user_program_visitor.pytamaro_python_used_functions,
)
print("----")
print("Excluded names: ", user_program_visitor.excluded_names)

{'DEF-function': 1, 'DEF-constant': 7, 'Python-op-div': 1, 'USE-constant': 29, 'rotate': 2, 'circular_sector': 1, 'Python-op-ifexp': 1, 'Python-op-and': 1, 'Python-op-eq': 1, 'Python-op-mod': 1, 'Python-op-not': 1, 'Python-op-lte': 1, 'Python-op-sub': 2, 'Python-op-add': 2, 'Python-op-usub': 1, 'int': 1, 'min': 1, 'Python-op-uadd': 1, 'Python-op-pow': 1, 'rgb_color': 1, 'show_graphic': 2, 'USE-function1': 3, 'beside': 1, 'above': 1, 'rectangle': 1, 'red': 2, 'triangle': 1}
----
User defined functions:  ['pacman']
----
PyTamaro/Python used functions:  {'rectangle', 'rotate', 'int', 'min', 'circular_sector', 'rgb_color', 'triangle', 'show_graphic', 'beside', 'above'}
----
Excluded names:  {'test_3', 'my_pacman', 'test', 'test_4', 'test_2', 'house', 'mouth_angle', 'pacman', 'magic_number', 'List'}


# Stiching together the TamaroCards

In [62]:
import svg_stack as ss
from lxml import etree

def create_svg_stack(tamaro_cards: dict[str, int], output_folder: str = ".") -> None:
    A4_WIDTH = 1200
    A4_HEIGHT = 800
    CARD_WIDTH = 200  # Assuming each card has a width of 100 units
    CARD_HEIGHT = 200  # Assuming each card has a height of 100 units

    page_number = 0
    svg = ss.Document()
    page_layout = ss.VBoxLayout()
    row_layout = ss.HBoxLayout()

    for card, count in tamaro_cards.items():
        for _ in range(count):
            if row_layout.get_size().width + CARD_WIDTH > A4_WIDTH:
                page_layout.addLayout(row_layout)
                row_layout = ss.HBoxLayout()
            if page_layout.get_size().height + CARD_HEIGHT > A4_HEIGHT:
                svg.setLayout(page_layout)
                svg.save(f"{output_folder}/tamaroCards_{page_number}.svg")
                page_layout = ss.VBoxLayout()
                page_number += 1

            # Read and modify the SVG file
            svg_path = f"cards/{card}.svg"
            svg_tree = etree.parse(svg_path)

            # Get the root element
            root = svg_tree.getroot()
            # Replace the width and height attributes
            root.attrib["width"] = str(CARD_WIDTH)
            root.attrib["height"] = str(CARD_HEIGHT)
            # Get the string representation of the modified SVG
            svg_string = etree.tostring(root).decode()
            row_layout.addSVG("empty_svg.svg", alignment=ss.AlignHCenter | ss.AlignVCenter, xml=svg_string)

    if row_layout.get_size().width > 0:
        page_layout.addLayout(row_layout)
    if page_layout.get_size().height > 0:
        svg.setLayout(page_layout)
        svg.save(f"{output_folder}/tamaroCards_{page_number}.svg")

create_svg_stack(user_program_visitor.tamaro_cards, output_folder="output")